In [82]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

In [3]:
df=pd.read_csv("/kaggle/input/nifty50-stocks-dataset/National_Stock_Exchange_of_India_Ltd.csv")

In [4]:
df.sample()

,Symbol,Open,High,Low,LTP,Chng,% Chng,Volume (lacs),Turnover (crs.),52w H,52w L,365 d % chng,30 d % chng
14,GRASIM,"1,757.30","1,757.85","1,679.00","1,685.80",-80.95,-4.58,7.48,127.84,"1,893.00",840.05,99.95,-3.08


In [5]:
import requests
import pandas as pd

# Replace 'YOUR_API_KEY' with your actual API key
api_key = 'YOUR_API_KEY'
symbol = 'AAPL'
url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval=1min&apikey={api_key}'

response = requests.get(url)
data = response.json()

# Extract time series data
timeseries = data.get('Time Series (1min)', {})

# Convert to DataFrame
df = pd.DataFrame.from_dict(timeseries, orient='index')
df = df.rename_axis('Timestamp').reset_index()

# Convert only numeric columns to float
numeric_cols = df.columns.difference(['Timestamp'])
df[numeric_cols] = df[numeric_cols].astype(float)

len(df) 


100

In [9]:
df.sample()

,Timestamp,1. open,2. high,3. low,4. close,5. volume
26,2024-08-09 19:33:00,216.32,216.32,216.31,216.315,298.0


In [21]:
# print(data)
fg=pd.DataFrame.from_dict(data,orient='index')
fg=fg.rename_axis("Timestamp").reset_index()
# fg_num=fg.columns.difference(["Timestamp"])
# fg[fg_num]=fg[fg_num].astype(float)
fg

,Timestamp,1. Information,2. Symbol,3. Last Refreshed,4. Interval,5. Output Size,6. Time Zone,2024-08-09 19:59:00,2024-08-09 19:58:00,2024-08-09 19:57:00,...,2024-08-09 18:29:00,2024-08-09 18:28:00,2024-08-09 18:27:00,2024-08-09 18:26:00,2024-08-09 18:25:00,2024-08-09 18:24:00,2024-08-09 18:23:00,2024-08-09 18:22:00,2024-08-09 18:21:00,2024-08-09 18:20:00
0,Meta Data,"Intraday (1min) open, high, low, close prices ...",AAPL,2024-08-09 19:59:00,1min,Compact,US/Eastern,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Time Series (1min),NaN,NaN,NaN,NaN,NaN,NaN,"{'1. open': '216.4100', '2. high': '216.4680',...","{'1. open': '216.4100', '2. high': '216.4700',...","{'1. open': '216.4100', '2. high': '216.4500',...",...,"{'1. open': '216.1700', '2. high': '216.2000',...","{'1. open': '216.1800', '2. high': '216.1800',...","{'1. open': '216.2000', '2. high': '216.2000',...","{'1. open': '216.1750', '2. high': '216.2000',...","{'1. open': '216.1800', '2. high': '216.2000',...","{'1. open': '216.1800', '2. high': '216.1800',...","{'1. open': '216.1500', '2. high': '216.1800',...","{'1. open': '216.2000', '2. high': '216.2000',...","{'1. open': '216.1100', '2. high': '216.1800',...","{'1. open': '216.1600', '2. high': '216.2000',..."


In [35]:
dj=data.get("Time Series (1min)",{})
dt=pd.DataFrame.from_dict(dj,orient='index')
dt=dt.rename_axis("TimeStamp").reset_index()
dr_num=dt.columns.difference(["TimeStamp"])
dt[dr_num]=dt[dr_num].astype(float)
dt.tail()

,TimeStamp,1. open,2. high,3. low,4. close,5. volume
95,2024-08-09 18:24:00,216.18,216.18,216.175,216.175,21.0
96,2024-08-09 18:23:00,216.15,216.18,216.150,216.180,111.0
97,2024-08-09 18:22:00,216.20,216.20,216.170,216.190,32.0
98,2024-08-09 18:21:00,216.11,216.18,216.110,216.175,432.0
99,2024-08-09 18:20:00,216.16,216.20,216.110,216.110,7.0


In [42]:
dt["TimeStamp"]=pd.to_datetime(dt["TimeStamp"])
dt["dayofweek"]=dt["TimeStamp"].dt.dayofweek
dt["Year"]=dt["TimeStamp"].dt.year
dt["Month"]=dt["TimeStamp"].dt.month
dt["day"]=dt["TimeStamp"].dt.day
dt["Hour"]=dt["TimeStamp"].dt.hour
dt["Minutes"]=dt["TimeStamp"].dt.minute
dt["Second"]=dt["TimeStamp"].dt.second

In [46]:
new_dt=dt.drop(columns=["TimeStamp"])
new_dt
X=new_dt.drop(columns=["4. close"])
Y=new_dt[["4. close"]]

In [53]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.1,random_state=42)

In [66]:
scaler=StandardScaler()
X_train_scaler=scaler.fit_transform(X_train)
X_test_scaler=scaler.transform(X_test)
# Y_train_scaler=scaler.transform(Y_train.reshape(-1,1))
# Y_test_scaler=scaler.transform(Y_test.reshape(-1,1))

In [59]:
random_forest=RandomForestRegressor()
random_forest.fit(X_train_scaler,Y_train)
random_forest.score(X_test_scaler,Y_test)

/tmp/ipykernel_33/2886111015.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  random_forest.fit(X_train_scaler,Y_train)


0.931322914287554

In [93]:
from sklearn.preprocessing import StandardScaler

# Assuming y_scaler was used to scale Y_train
y_scaler12 = StandardScaler()
Y_train_scaled12 = y_scaler12.fit_transform(Y_train.values.reshape(-1, 1))

# Model prediction
y_linear_pred12 = random_forest.predict(X_test_scaler).reshape(-1, 1)

# Inverse transform using y_scaler
scaler_inverse_linear12 = y_scaler12.inverse_transform(y_linear_pred12)
scaler_inverse_linear12

array([[233.17730636],
       [233.18908469],
       [233.17940949],
       [233.18761093],
       [233.18705818],
       [233.18861168],
       [233.18800263],
       [233.17812728],
       [233.19290395],
       [233.19580065]])

In [83]:
linear_reg=LinearRegression()
linear_reg.fit(X_train_scaler,Y_train)

LinearRegression()

In [84]:
linear_reg.score(X_test_scaler,Y_test)

0.9010361591940372

In [91]:
scaler_inverse_linear1

array([[233.17714585],
       [233.18932509],
       [233.17947369],
       [233.18735511],
       [233.18682698],
       [233.18994258],
       [233.18810526],
       [233.17799165],
       [233.19319013],
       [233.19824074]])